# Notes

## Flight paths:
- **Houston, TX** to **Los Angeles, CA** (IAH - LAX)
- **New York City, NY** to **Miami, FL** (JFK - MIA)
- **Portland, WA** to **Chicago, IL** (PDX - ORD)

## Number of Total Routes
- at least 1,000 per route for now (All times in GMC)

    -__Times:__

        - 0000 hours to 0600 hours

        - 0601 hours to 1200 hours

        - 1201 hours to 1800 hours

        - 1801 hours to 2399 hours

    - __Times of the Year:__
    
        - Try to get every month

    - __Times of the Week:__
        - Try to get every day. 
    
## Notes about Data considerations

- see what are typical flight times for your paths. you may be limited here. 

- consistently work with other api to see what you can grab

- grab future flights too!

- do some division on how many flights you can grab from how many time zonestuff


## Features to Scrape
- want aircraft type
- want airline flight info
- want airline flight 
- want aircraft type struct
- want flight struct

1 query will generate 15 results. Ex; If you request to see all flight from airport Alpha to Airport Bravo and the search results come back with 5000 flights. To find the pricing estimate you would do the following math. 5000/15 = 333 * $0.0079 = $2.63 (Class 2)

In [269]:
import sys
from suds import null, WebFault
from suds.client import Client
import logging
import json
import pandas as pd
import datetime
import numpy as np

In [2]:
with open('/Users/ChristopherKuzemka/Documents/GA/dsi_11/projects/capstone/env.json') as f:
    information = json.load(f)

In [3]:
information.keys()

dict_keys(['FA_API_KEY', 'FA_USERNAME', 'x-rapidapi-host', 'x-rapidapi-key'])

In [4]:
username = information.get('FA_USERNAME')
apiKey = information.get('FA_API_KEY')
url = 'http://flightxml.flightaware.com/soap/FlightXML2/wsdl'

In [5]:
logging.basicConfig(level=logging.INFO)
api = Client(url, username=username, password=apiKey)

## Testing playground determining usefulness of API

In [83]:
#Get Aircraft type  - semi-useful
aircraft_type = api.service.AircraftType('B744')
aircraft_type

(AircraftTypeStruct){
   manufacturer = "Boeing"
   type = "747-400"
   description = "quad-jet"
 }

# JFK - MIA

From [here](https://www.flights.com/flights/new-york-jfk-to-miami-mia/): "with 3 differnt airlines operating flights between New York and Miami, there are, on average, 2,197 flights per month.. This equates to about 523 flights per week, and 75 flights per day from JFK to MIA. The three airlines are:
- American Airlines (Flight AA 2572)

- British Airways (Flight BA 1687) 

- Malaysia Airlines (Flight MH 9446).

In [7]:
def make_unix_lists(start_date, end_date, frequency):
    created_range = pd.date_range(start = start_date, end = end_date, freq = frequency) #creates a daterange series
    list_created_range = list(created_range) #converts such range into a list
    unix_floats = [date.to_pydatetime().timestamp() for date in list_created_range] #transforms the daterange list into unix epoch tiimestamps represented as floats
    unix_ints = [int(i) for i in unix_floats] #makes the above list as a list of integers
    #return unix_ints

    #We are doing the below to accomodate a for loop format into another function
    start_ints = unix_ints[:-1] #creates a list of all the start dates without last element
    end_ints = unix_ints[1:] #creates a list of all the end dates without first element 

    return start_ints, end_ints

In [8]:
start, end = make_unix_lists('5/11/2020', '5/24/2020', '8H')

In [9]:
start

[1589169600,
 1589198400,
 1589227200,
 1589256000,
 1589284800,
 1589313600,
 1589342400,
 1589371200,
 1589400000,
 1589428800,
 1589457600,
 1589486400,
 1589515200,
 1589544000,
 1589572800,
 1589601600,
 1589630400,
 1589659200,
 1589688000,
 1589716800,
 1589745600,
 1589774400,
 1589803200,
 1589832000,
 1589860800,
 1589889600,
 1589918400,
 1589947200,
 1589976000,
 1590004800,
 1590033600,
 1590062400,
 1590091200,
 1590120000,
 1590148800,
 1590177600,
 1590206400,
 1590235200,
 1590264000]

In [10]:
jfk_mia_info = {'AA':'2572',
                'BA':'1687',
                'MA':'9446'}

In [338]:
def make_schedule_df(start_input, end_input, frequency_input, origin_input, destination_input, flight_info_input, howMany_input):
    ## Empty lists for storage reasons
    schedules = [] #instantiates the schedules column
    modified_schedules = [] #instantiates the modified schedules list
    flight_info_list = [] #instantiates the flight info list


    ## Creates the sketelton dataframe for schedules
    schedules_cols = ['ident', 'actual_ident', 'departuretime', 'arrival_time', 'origin', 'destination', 'aircrafttype', 'meal_service', 'seats_cabin_first', 'seats_cabin_business', 'seats_cabin_coach'] #column names for the scheduling df
    schedules_df = pd.DataFrame(columns = column_names) #creates the empty schedules dataframe skeleton

    ## Creates the skeleton dataframe for the flight_info_df
    flight_info_cols = ['faFlightID', 'ident', 'aircrafttype', 'filed_ete', 'filed_time', 'filed_departuretime', 'filed_airspeed_mach', 'filed_altitude', 'actualdeparturetime', 'estimatedarrivaltime', 'actualarrivaltime', 'diverted', 'origin', 'originName', 'originCity', 'destinationName', 'destinationCity']
    flight_info_df = pd.DataFrame(columns = names) #creates the empty flight_info dataframe skeleton

    ## Unix Epoch Time Function
    start_list, end_list = make_unix_lists(start_input, end_input, frequency_input) #utilizes the unix epoch creation list function

    ## Magic happens here
    for airline_code in flight_info_input: #for the keys in the flight info input 
        for j in range(len(start_list)): #for the entirity of the unix epoch time list
            airline_flight_schedules = api.service.AirlineFlightSchedules(startDate = start_list[j], endDate = end_list[j], origin = origin_input, destination = destination_input, airline = airline_code, flightno = flight_info_input.get(airline_code), howMany = howMany_input) #gets the flight schedules information

            #Credit for the dict below
            #https://stackoverflow.com/questions/17581731/parsing-suds-soap-complex-data-type-into-python-dict
            airline_flight_dict = Client.dict(airline_flight_schedules) #converts the above variable to a dictionary
            schedules.append(airline_flight_dict) #appends results to the instantiated output list
    
    ## Cleans the dirty collection for schedules
    for k in range(len(schedules)):
        try: #try the below...we'll get some errors for empty stuff
            modified_schedules.append(schedules[k]['data']) #filters the output list to only show meaningful information where actual flights were flows in a timeframes studied
        except: #when an error occurs...
            continue #..move on and ignore any issues with with non existent flight data

    ## Creates a dataframe out of the modified_schedules list
    for l in range(len(modified_schedules)):
        for m in range(len(modified_schedules[l])):
            sched_df = pd.DataFrame(modified_schedules[l][m]).T #uses a transposed version to cleanly extract the data
            sched_df.rename(columns = {0:schedules_cols[0], #renames the columns to match the end_df so concatenation is simple
                        1: schedules_cols[1],
                        2: schedules_cols[2],
                        3: schedules_cols[3],
                        4: schedules_cols[4],
                        5: schedules_cols[5],
                        6: schedules_cols[6],
                        7: schedules_cols[7],
                        8: schedules_cols[8],
                        9: schedules_cols[9],
                        10: schedules_cols[10]}, inplace = True) #very inefficient way to rename my columns lolololol
            schedules_df = pd.concat([schedules_df, sched_df]) #overwrites the end_df to create a larger dataframe
    
    ## Cleans the schedules_df
    schedules_df.drop(index = 0, inplace = True) #drops the 0 index rows in the repetative 0,1 pattern of the dataframe
    schedules_df.reset_index(inplace = True) #resets the index by creating a new index row
    schedules_df.drop(columns = 'index', inplace = True) #drops the extra 'index row


    ## Gets the flight information list using the schedules_df idents
    for identifier in schedules_df['ident']: #calls to each identifier collected
        for offset_number in range(20): #set in place to not miss any flights in 2 week period 
            try: #we're gonna get some errors here for empty flights I think
                flight_info_ex = api.service.FlightInfoEx(ident = identifier, howMany = 15, offset = offset_number) #retrieves the information pertaining to the identifier
                flight_info = Client.dict(flight_info_ex) #converts the suds object into a workable list/dictionary??
                flight_info_list.append(flight_info_ex) #appends whatever the above is into a list
            except: #when an error occurs...
                continue #...just move on

    ## Creates a dataframe out of the flight_info_df
    for query in flight_info_list: #calls to every query in the info list made
        for flight in query['flights']: #for every flight in the dictionary
            flight_dict = {} #empty dicitonary pertaining to the columns in each flight 
            for column in flight_info_cols: #for every column name in the flight_info_cols
                flight_dict[column] = flight[column] #appends the column names to the dictionary
            info_df = pd.DataFrame([flight_dict], columns=flight_dict.keys()) #makes a dataframe using the dictionary columns 
            flight_info_df = pd.concat([flight_info_df, info_df], axis =0).reset_index(drop = True) #resets the index and fixes all those issues

    return schedules_df, flight_info_df

In [339]:
jfk_mia_flight_scheds, jfk_mia_flight_info = make_schedule_df('5/12/2020', '5/25/2020', '8H', 'JFK', 'MIA', jfk_mia_info, 15)

ERROR:suds.client:<suds.sax.document.Document object at 0x1223de790>
ERROR:suds.client:<suds.sax.document.Document object at 0x121f33c10>
ERROR:suds.client:<suds.sax.document.Document object at 0x120a07c50>
ERROR:suds.client:<suds.sax.document.Document object at 0x122de4a10>
ERROR:suds.client:<suds.sax.document.Document object at 0x1220fb1d0>
ERROR:suds.client:<suds.sax.document.Document object at 0x12207d450>
ERROR:suds.client:<suds.sax.document.Document object at 0x1222a5910>
ERROR:suds.client:<suds.sax.document.Document object at 0x122d41d90>
ERROR:suds.client:<suds.sax.document.Document object at 0x12277b250>
ERROR:suds.client:<suds.sax.document.Document object at 0x1229a3690>
ERROR:suds.client:<suds.sax.document.Document object at 0x1229a3f50>
ERROR:suds.client:<suds.sax.document.Document object at 0x1221f5950>
ERROR:suds.client:<suds.sax.document.Document object at 0x1226c0c90>
ERROR:suds.client:<suds.sax.document.Document object at 0x122437d90>
ERROR:suds.client:<suds.sax.docume

In [340]:
jfk_mia_flight_scheds

,ident,actual_ident,departuretime,arrival_time,origin,destination,aircrafttype,meal_service,seats_cabin_first,seats_cabin_business,seats_cabin_coach
0,MAS9446,AAL2572,1589308200,1589320080,KJFK,KMIA,B738,Business: Meal / Economy: Meal,0,16,144
1,AAL2572,None,1589310000,1589322000,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
2,MAS9446,AAL2572,1589394600,1589406480,KJFK,KMIA,B738,Business: Meal / Economy: Meal,0,16,144
3,AAL2572,None,1589396400,1589408400,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
4,GLO6391,AAL2572,1589396400,1589408400,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
5,MAS9446,AAL2572,1589481000,1589492880,KJFK,KMIA,B738,Business: Meal / Economy: Meal,0,16,144
6,GLO6391,AAL2572,1589482800,1589494800,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
7,AAL2572,None,1589482800,1589494800,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144
8,MAS9446,AAL2572,1589567400,1589579280,KJFK,KMIA,B738,Business: Meal / Economy: Meal,0,16,144
9,GLO6391,AAL2572,1589569200,1589581200,KJFK,KMIA,B738,Business: No meal / Economy: No meal,0,16,144


In [341]:
jfk_mia_flight_info

,faFlightID,ident,aircrafttype,filed_ete,filed_time,filed_departuretime,filed_airspeed_mach,filed_altitude,actualdeparturetime,estimatedarrivaltime,actualarrivaltime,diverted,origin,originName,originCity,destinationName,destinationCity
0,AAL2572-1590381915-airline-0034,AAL2572,B738,02:58:00,1590381915,1590604800,None,0,0,1590615480,0,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
1,AAL2572-1590295537-airline-0339,AAL2572,B738,02:33:00,1590295537,1590518400,None,380,0,1590528840,0,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
2,AAL2572-1590209159-airline-0537,AAL2572,B738,03:06:00,1590431725,1590432000,None,360,1590431773,1590440887,1590440887,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
3,AAL2572-1590122721-airline-0420,AAL2572,B738,02:40:00,1590345342,1590345600,None,380,1590345390,1590354360,1590354360,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
4,AAL2572-1590036344-airline-0175,AAL2572,B738,02:55:00,1590259235,1590259200,None,380,1590259283,1590267480,1590267480,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880,AAL2572-1589345159-airline-0102,AAL2572,B738,02:56:00,1589567883,1589568000,None,380,1589567931,1589576820,1589576820,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
3881,AAL2572-1589258723-airline-0379,AAL2572,B738,02:51:00,1589481174,1589481600,None,380,1589481222,1589489820,1589489820,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
3882,AAL2572-1589345159-airline-0102,AAL2572,B738,02:56:00,1589567883,1589568000,None,380,1589567931,1589576820,1589576820,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"
3883,AAL2572-1589258723-airline-0379,AAL2572,B738,02:51:00,1589481174,1589481600,None,380,1589481222,1589489820,1589489820,None,KJFK,John F Kennedy Intl,"New York, NY",Miami Intl,"Miami, FL"


In [316]:
output = []
for i in jfk_mia_flight_scheds['ident']:
    for j in range(20): 
        try:
            flight_info_ex = api.service.FlightInfoEx(ident = i, howMany = 15, offset = j)
            flight_inf = Client.dict(flight_info_ex)
            output.append(flight_info_ex)
        except:
            continue

ERROR:suds.client:<suds.sax.document.Document object at 0x120a37e10>
ERROR:suds.client:<suds.sax.document.Document object at 0x120935310>
ERROR:suds.client:<suds.sax.document.Document object at 0x121105bd0>
ERROR:suds.client:<suds.sax.document.Document object at 0x117643dd0>
ERROR:suds.client:<suds.sax.document.Document object at 0x1205de390>
ERROR:suds.client:<suds.sax.document.Document object at 0x120152310>
ERROR:suds.client:<suds.sax.document.Document object at 0x120165710>
ERROR:suds.client:<suds.sax.document.Document object at 0x120165d50>
ERROR:suds.client:<suds.sax.document.Document object at 0x11f303050>
ERROR:suds.client:<suds.sax.document.Document object at 0x11f300710>
ERROR:suds.client:<suds.sax.document.Document object at 0x11f300750>
ERROR:suds.client:<suds.sax.document.Document object at 0x11f307250>
ERROR:suds.client:<suds.sax.document.Document object at 0x11ff714d0>
ERROR:suds.client:<suds.sax.document.Document object at 0x11fcf0190>
ERROR:suds.client:<suds.sax.docume

In [328]:
unique_length = []
for i in range(len(output)):
    #for j in range(len(output[i])):
        #print(len(output[i]))
    if not output[i][0] in unique_length:
        unique_length.append(output[i][0])

print(unique_length)

[-1]


In [329]:
output[0][1]

[(FlightExStruct){
    faFlightID = "AAL2572-1590381915-airline-0034"
    ident = "AAL2572"
    aircrafttype = "B738"
    filed_ete = "02:58:00"
    filed_time = 1590381915
    filed_departuretime = 1590604800
    filed_airspeed_kts = 321
    filed_airspeed_mach = None
    filed_altitude = 0
    route = None
    actualdeparturetime = 0
    estimatedarrivaltime = 1590615480
    actualarrivaltime = 0
    diverted = None
    origin = "KJFK"
    destination = "KMIA"
    originName = "John F Kennedy Intl"
    originCity = "New York, NY"
    destinationName = "Miami Intl"
    destinationCity = "Miami, FL"
  },
 (FlightExStruct){
    faFlightID = "AAL2572-1590295537-airline-0339"
    ident = "AAL2572"
    aircrafttype = "B738"
    filed_ete = "02:33:00"
    filed_time = 1590295537
    filed_departuretime = 1590518400
    filed_airspeed_kts = 373
    filed_airspeed_mach = None
    filed_altitude = 380
    route = "WAVEY EMJAY J174 SWL CEBEE WETRO DIW AR22 JORAY HILEY7"
    actualdeparturetime 

In [330]:
for query in output:
    for flight in query['flights']:
        f_time = flight['filed_time']
        print(f_time) 

5
1589827142
1589741446
1589654365
1589567883
1589481174
1590000375
1589914315
1589827142
1589741446
1589654365
1589567883
1589481174
1589914315
1589827142
1589741446
1589654365
1589567883
1589481174
1589827142
1589741446
1589654365
1589567883
1589481174
1589741446
1589654365
1589567883
1589481174
1589654365
1589567883
1589481174
1589567883
1589481174
1589481174
1590381915
1590295537
1590431725
1590345342
1590259235
1590172660
1590086190
1590000375
1589914315
1589827142
1589741446
1589654365
1589567883
1589481174
1590295537
1590431725
1590345342
1590259235
1590172660
1590086190
1590000375
1589914315
1589827142
1589741446
1589654365
1589567883
1589481174
1590431725
1590345342
1590259235
1590172660
1590086190
1590000375
1589914315
1589827142
1589741446
1589654365
1589567883
1589481174
1590345342
1590259235
1590172660
1590086190
1590000375
1589914315
1589827142
1589741446
1589654365
1589567883
1589481174
1590259235
1590172660
1590086190
1590000375
1589914315
1589827142
1589741446
15896543

In [331]:
output[0]['flights']

[(FlightExStruct){
    faFlightID = "AAL2572-1590381915-airline-0034"
    ident = "AAL2572"
    aircrafttype = "B738"
    filed_ete = "02:58:00"
    filed_time = 1590381915
    filed_departuretime = 1590604800
    filed_airspeed_kts = 321
    filed_airspeed_mach = None
    filed_altitude = 0
    route = None
    actualdeparturetime = 0
    estimatedarrivaltime = 1590615480
    actualarrivaltime = 0
    diverted = None
    origin = "KJFK"
    destination = "KMIA"
    originName = "John F Kennedy Intl"
    originCity = "New York, NY"
    destinationName = "Miami Intl"
    destinationCity = "Miami, FL"
  },
 (FlightExStruct){
    faFlightID = "AAL2572-1590295537-airline-0339"
    ident = "AAL2572"
    aircrafttype = "B738"
    filed_ete = "02:33:00"
    filed_time = 1590295537
    filed_departuretime = 1590518400
    filed_airspeed_kts = 373
    filed_airspeed_mach = None
    filed_altitude = 380
    route = "WAVEY EMJAY J174 SWL CEBEE WETRO DIW AR22 JORAY HILEY7"
    actualdeparturetime 

In [332]:
print(output)

itude = 380
         route = "WAVEY EMJAY J174 ORF J121 CHS CAKET Q97 KENLL OMN HILEY7"
         actualdeparturetime = 1589741494
         estimatedarrivaltime = 1589750762
         actualarrivaltime = 1589750762
         diverted = None
         origin = "KJFK"
         destination = "KMIA"
         originName = "John F Kennedy Intl"
         originCity = "New York, NY"
         destinationName = "Miami Intl"
         destinationCity = "Miami, FL"
      },
      (FlightExStruct){
         faFlightID = "AAL2572-1589431536-airline-0436"
         ident = "AAL2572"
         aircrafttype = "B738"
         filed_ete = "02:40:00"
         filed_time = 1589654365
         filed_departuretime = 1589654400
         filed_airspeed_kts = 465
         filed_airspeed_mach = None
         filed_altitude = 300
         route = "WAVEY EMJAY J174 ZIZZI J174 ORF J121 CHS CAKET Q97 KENLL OMN HILEY7"
         actualdeparturetime = 1589654430
         estimatedarrivaltime = 1589663460
         actualarriva

In [334]:
for query in output:
  for flight in query['flights']:
    flight_dict = {}
    for column in names:
        flight_dict[column] = flight[column]
    df  = pd.DataFrame([flight_dict], columns=flight_dict.keys())
    some_df = pd.concat([some_df, df], axis =0).reset_index(drop = True)
    

In [335]:
some_df.shape

(3885, 17)

In [ ]:
df  = pd.DataFrame([podcast_dict], columns=podcast_dict.keys())
df_podcast = pd.concat([df_podcast, df], axis =0).reset_index()

In [79]:
flight_info_ex = api.service.FlightInfoEx(ident = jfk_mia_flight_scheds[0][1]['ident'], howMany = 15, offset = 0)
flight_info_ex

(FlightInfoExStruct){
   next_offset = -1
   flights[] = 
      (FlightExStruct){
         faFlightID = "AAL2572-1590381915-airline-0034"
         ident = "AAL2572"
         aircrafttype = "B738"
         filed_ete = "02:58:00"
         filed_time = 1590381915
         filed_departuretime = 1590604800
         filed_airspeed_kts = 321
         filed_airspeed_mach = None
         filed_altitude = 0
         route = None
         actualdeparturetime = 0
         estimatedarrivaltime = 1590615480
         actualarrivaltime = 0
         diverted = None
         origin = "KJFK"
         destination = "KMIA"
         originName = "John F Kennedy Intl"
         originCity = "New York, NY"
         destinationName = "Miami Intl"
         destinationCity = "Miami, FL"
      },
      (FlightExStruct){
         faFlightID = "AAL2572-1590295537-airline-0339"
         ident = "AAL2572"
         aircrafttype = "B738"
         filed_ete = "02:58:00"
         filed_time = 1590295537
         filed_dep

In [47]:
flight_inf = Client.dict(flight_info_ex)

In [52]:
flight_inf['flights'][0]['faFlightID']

AAL2572-1590295537-airline-0339

In [71]:
#this gets me a list of flight information like speed and altitude I believe this will be enough must convert this to a dataframe
get_historical_track = api.service.GetHistoricalTrack(flight_inf['flights'][0]['faFlightID'])

ERROR:suds.client:<suds.sax.document.Document object at 0x1182a3710>


WebFault: Server raised fault: 'NO_DATA no data available'

In [70]:
get_historical_track[0]

dspeed = 456
    altitude = 360
    altitudeStatus = "-"
    updateType = "A"
    altitudeChange = "-"
  },
 (TrackStruct){
    timestamp = 1589660920
    latitude = 29.69165
    longitude = -81.12299
    groundspeed = 454
    altitude = 360
    altitudeStatus = "-"
    updateType = "A"
    altitudeChange = "-"
  },
 (TrackStruct){
    timestamp = 1589660951
    latitude = 29.62766
    longitude = -81.12342
    groundspeed = 453
    altitude = 360
    altitudeStatus = "-"
    updateType = "A"
    altitudeChange = "-"
  },
 (TrackStruct){
    timestamp = 1589660981
    latitude = 29.56384
    longitude = -81.12347
    groundspeed = 451
    altitude = 360
    altitudeStatus = "-"
    updateType = "A"
    altitudeChange = "-"
  },
 (TrackStruct){
    timestamp = 1589661011
    latitude = 29.50076
    longitude = -81.12084
    groundspeed = 451
    altitude = 360
    altitudeStatus = "-"
    updateType = "A"
    altitudeChange = "-"
  },
 (TrackStruct){
    timestamp = 1589661041
    latit

In [130]:
data = [Client.dict(suds_object) for suds_object in get_historical_track[0]]

In [131]:
df = pd.DataFrame(data)

In [132]:
df.head()

,timestamp,latitude,longitude,groundspeed,altitude,altitudeStatus,updateType,altitudeChange
0,1589396232,40.61018,-73.79260,170,10,-,A,-
1,1589396248,40.59851,-73.80048,183,14,-,A,-
2,1589396264,40.58582,-73.80909,196,16,-,A,-
3,1589396281,40.57109,-73.81738,218,20,-,A,-
4,1589396297,40.55347,-73.81714,241,22,-,A,-


In [137]:
df.shape

(322, 8)

In [133]:
df['flight_identifier'] = 'aasjhdfhj'

In [134]:
df.head()

,timestamp,latitude,longitude,groundspeed,altitude,altitudeStatus,updateType,altitudeChange,flight_identifier
0,1589396232,40.61018,-73.79260,170,10,-,A,-,aasjhdfhj
1,1589396248,40.59851,-73.80048,183,14,-,A,-,aasjhdfhj
2,1589396264,40.58582,-73.80909,196,16,-,A,-,aasjhdfhj
3,1589396281,40.57109,-73.81738,218,20,-,A,-,aasjhdfhj
4,1589396297,40.55347,-73.81714,241,22,-,A,-,aasjhdfhj


# IAH - LAX


From [here](https://www.flights.com/flights/houston-iah-to-los-angeles-lax/): "With 6 different airlines operating between houstan and Los Angeles, there are, on average, 2,642 flights per month. This equates to about 629 flights per week, and 90 flights per day from IAH to LAX." 5 out of 6 of these airliners are: 
- United Airlines (Flight UA 302) 
- Air China (CA 7398), Azul (AD 7026)
- Asiana Airlines (OZ 9512)
- Air New Zealand (NZ 9653)